<a href="https://colab.research.google.com/github/Abhinaay/Hamara-Paryatan-Katai-Jahar/blob/master/project_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!chmod +x detection.py

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      //const capture = document.createElement('button');
      //capture.textContent = 'Capture';
      //div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => 1);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [40]:

import numpy as np
import tensorflow as tf
import cv2
import getch
import time
from IPython.display import Image


#video=take_photo #this variable either contain path to the video for processing or either camera input
ranger=3 #This variable defines no of which must increase or decrease in the frame to change the output

class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.compat.v1.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        #print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

if __name__ == "__main__":
    model_path = 'frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.7
    #cap = take_photo()
    print("Press ESC key to exit the program ")
    print("\n\n")
    #choose=input("Visual Output Required(y) or not(n) ")
    old=0
    print("No of Humans are: 0")
    #if choose.lower()=='y':
    while True:
        img = take_photo()
        img = cv2.imread('photo.jpg')
        img = cv2.resize(img, (1280, 720))

        boxes, scores, classes, num = odapi.processFrame(img)
            
            # Visualization of the results of a detection
        count=0
        for i in range(len(boxes)):
                # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,255),2)
                count+=1
            if old==0 and count>0:
                print("No of Humans are:",count)
                old=count
            elif count>=old+ranger:
                print("No of Humans are:",count)
                old=count
            elif count<old:
                print("No of Humans are:",count)
                old=count
            #display("preview", Image(img))
            if cv2.waitKey(0)==13:
                #if ord(getch.getch()) == 13:
                break
    cv2.destroyAllWindows()
    cap.release()
    print("\n\n\tBYEBYE\n\n")
'''
    elif choose.lower()=='n':
        while True:
            img = take_photo()
            img = cv2.imread('photo.jpg')
            img = cv2.resize(img, (1280, 720))

            boxes, scores, classes, num = odapi.processFrame(img)
            count=0
            for i in range(len(boxes)):
                if classes[i] == 1 and scores[i] > threshold:
                    count+=1
            if old==0 and count>0:
                print("No of Humans are:",count)
                old=count
            elif count>=old+ranger:
                print("No of Humans are:",count)
                old=count
            elif count<old:
                print("No of Humans are:",count)
                old=count
            if cv2.waitKey(0)==13:
                break
'''                
                  





Press ESC key to exit the program 



No of Humans are: 0


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored